# Sistemas Inteligentes 2021/2022

## Mini-projeto 2: Quadrados Latinos


## Grupo: 10

### Elementos do Grupo

Número: 59270 Nome: Tatiana Moura
Número: 55945 Nome: Mariana Valente
Número: 54356 Nome: Judite Ramos      

(Nota: Neste relatório pode adicionar as células de texto e código que achar necessárias.)

## Representação de variáveis, domínios, vizinhos e restrições

Descreva aqui, textualmente, como decidiu representar as variáveis, domínios, vizinhos e restrições em Python;

**variáveis -->** Representadas por uma lista de strings por exemplo:  [A1, A2, A3, B1, B2, B3, C1 ,C2 ,C3], cada variável corresponde a uma posição na matriz.

**domínios -->** Representados por um dicionário em que as chaves são as variáveis e os valores são os possíveis números de 1 a n, sendo que n é o tamanho da matriz.

**vizinhos -->** Representados por um dicionário em que as chaves são as variáveis e os valores são a lista de variáveis com a qual tem restrições.

**restrições -->** Na primeira parte do projeto usamos uma função do tipo f(A, a, B, b), que devolve True caso a e b sejam diferentes. Na segunda fase do projeto, a versão futoshiki, utilizamos também uma função do tipo f(A, a, B, b), que devolve True caso as desigualdades sejam satisfeitas.

## Formulação do problema

In [92]:
from csp import *

alfabeto='ABCDEFGHIJKLMNOPQRSTUVWXYZ'
def quadrado_latino(len = 4, dom_quadrado = {}, ineq = []):
    """
    Pode receber parametros ou não.
    Deve devolver um CSP, à semelhança dos guiões das aulas PL.
    Comente o código.
    """
    #definir as variáveis
    var = []
    for x in range(len):
        acc = x+1 #o x começa no zero, e as nossas variáveis começam no a1, não no a0
        letras = [alfabeto[n] + str(acc) for n in range(len)] #para termos o numero de letras correspondente a n (se n = 4, letras vão até D), juntamente com os números obtidos anteriormente
        var = var + letras #juntar as letras com os numeros numa lista
          
    #definir domínios     
    dominios = {}
    if dom_quadrado != {}: #dicionário não vazio, para conseguir entrar no 'for' do dom_quadrado. Se estiver vazio, não vai fazer o for
        for v in var:
            for v2 in dom_quadrado: #2 ciclos 'for', para comparar a lista das variáveis com o dicionário das variáveis com valores atribuídos
                dominios[v2] = [dom_quadrado.get(v2)] #se a variável existir no dom_quadrado, o valor que ela tem(o seu domínio) é o que lhe está atribuído nesse dicionário
                if (v2[0] == v[0] or v2[1] == v[1]) and v2!= v: #se houver uma variável cujo primeiro elemento(a letra) é igual à letra de uma variável no dom_quadrado
                    #ou tem o segundo elemento(o número) igual a uma variável no dom_quadrado (e não são a mesma variável)
                    proibido = [dom_quadrado.get(v2)] #nesta lista fica o valor(domínio) que a variável presente no dom_quadrado tem
                    if v in dominios: #se a variável(que não está no dom_quadrado) já existir no dominios (já tem domínio)
                        dominios[v] = [x for x in range (1, len+1) if x not in proibido if x in dominios.get(v)] #o domínio da variável v é o conjunto de todos os números naturais
                        #até ao número de linhas/colunas do problema, exceto aquele que a variável no dom_quadrado (cuja letra ou número é igual à letra/número da variável v) tem. 
                        #Para além disso, não pode ser nenhum valor que a variável v não tenha já no domínio (se o valor não está no domínio, é devido a alguma restrição anterior)
                    else: #a variável v não tem ainda domínio atribuido 
                        dominios[v] = [x for x in range (1, len+1) if x not in proibido] #então a única restrição dos valores do seu domínio é não ser igual ao valor que a variável em
                        #dom_quadrado (que tem letra ou número igual a ela - isto é, pertence à mesma linha ou coluna) tem
                else: #a variável v não está na mesma linha nem coluna (não tenha letra ou número igual) a nenhuma variável do dom_quadrado
                    if v in dominios: #tudo igual aos pontos anteriores, exceto que não há a preocupação de ser o mesmo valor de uma variável presente em dom_quadrado
                        dominios[v] = [x for x in range (1, len+1) if x in dominios.get(v)]
                    else:
                        dominios[v] = [x for x in range (1, len+1)]
    else: #o dicionário está vazio, não há variaveis com valores atribuidos
        for v in var:
            dominios[v] = [x for x in range (1, len+1)] #os dominios são simplesmente todos os numeros naturais até ao nº de linhas do problema
    
    #definir vizinhos com restrições:
    visitadas = []
    vizinho = ''
    
    for v1 in var:
        vizinho += v1 + ':' #a função parse_neighbors requer uma string na forma 'X: Y Z; Y: Z', daí a variável ser seguida de ':'
        visitadas.append (v1) #necessário manter registo das variáveis já tidas em conta, uma vez que se declararmos X:Y na string, não precisamos de declarar Y:X
        for v2 in var:
            if v1 != v2 and (v1[0] == v2[0] or v1[1] == v2[1]) and v2 not in visitadas: #se as variáveis estão na mesma linha ou coluna
                #(têm letra ou número igual)
                vizinho += ' ' + v2 #então a seguir aos ':' adiciona-se a segunda variável, uma vez que os elementos na mesma linha ou coluna
                #vão ter restrições (definição de vizinho)
        vizinho += '; ' #para garantir que a semântica da string está como é pedida
    
    vizinho = vizinho[:-2] #é necessário retirar o último ';', para a função parse_neighbors funcionar
    vizinhos_quad = parse_neighbors(vizinho) #gerar os vizinhos num dicionário
    
    #definir restrições:
    
    def valores_diferentes(a, b):
        return a != b #os seus valores têm de ser diferentes
    
    #-------------------------------para o futoshiki------------------------------------------------------------
    
    def menor_que(a, b):
        return a < b
    
    def maior_que(a, b):
        return a > b
    
    def restricoes_quad(A,a,B,b):
        restricoes = {}
    
        if ineq == []:
            if (A[0] == B[0] or A[1] == B[1]) and A!=B:
                restricoes[(A,B)] = valores_diferentes
        
        else:
            desigualdades = []
        
            for i in ineq:
                i = i.split(' ')
                desigualdades.append(i)
        
            for d in desigualdades:
                if d[1] == '<' and d[0] == A and d[2]==B :
                    restricoes[(A,B)] = menor_que 
                if d[1] == '<' and d[0] == B and d[2]==A:
                    restricoes[(A,B)] = maior_que
                if d[1] == '>' and d[0] == A and d[2]==B :
                    restricoes[(A,B)] = maior_que
                if d[1] == '>' and d[0] == B and d[2]==A :
                    restricoes[(A,B)] = menor_que
                    
            if (A,B) not in restricoes:
                if (A[0] == B[0] or A[1] == B[1]) and A!=B:
                    restricoes[(A,B)] = valores_diferentes
    
        if (A,B) in restricoes :
            return restricoes[(A,B)](a,b)
        else:
            return True
        
    return CSP(var, dominios, vizinhos_quad, restricoes_quad)

## Criação do problema do quadrado latino simples

Mostrem que o código está a funcionar, construindo um problema de quadrado latino *4x4*, imprimindo as variáveis, domínios iniciais, e vizinhos. Adicione os comentários necessários. Mostre como podemos criar um puzzle com quadrados já preenchidos, e qual o impacto que isso tem nas variáveis, domínios iniciais, e vizinhos.

**Problema do quadrado latino vázio no inicio**

Neste exemplo concluímos que quando o quadrado está vázio, temos os domínios de 1 a 4 para todas a variáveis.

In [93]:
p = quadrado_latino(len=4)
print ('variáveis:', p.variables, '\n')
print ('domínios iniciais:', p.domains, '\n')
print ('vizinhos:', p.neighbors, '\n')

variáveis: ['A1', 'B1', 'C1', 'D1', 'A2', 'B2', 'C2', 'D2', 'A3', 'B3', 'C3', 'D3', 'A4', 'B4', 'C4', 'D4'] 

domínios iniciais: {'A1': [1, 2, 3, 4], 'B1': [1, 2, 3, 4], 'C1': [1, 2, 3, 4], 'D1': [1, 2, 3, 4], 'A2': [1, 2, 3, 4], 'B2': [1, 2, 3, 4], 'C2': [1, 2, 3, 4], 'D2': [1, 2, 3, 4], 'A3': [1, 2, 3, 4], 'B3': [1, 2, 3, 4], 'C3': [1, 2, 3, 4], 'D3': [1, 2, 3, 4], 'A4': [1, 2, 3, 4], 'B4': [1, 2, 3, 4], 'C4': [1, 2, 3, 4], 'D4': [1, 2, 3, 4]} 

vizinhos: defaultdict(<class 'list'>, {'A1': ['B1', 'C1', 'D1', 'A2', 'A3', 'A4'], 'B1': ['A1', 'C1', 'D1', 'B2', 'B3', 'B4'], 'C1': ['A1', 'B1', 'D1', 'C2', 'C3', 'C4'], 'D1': ['A1', 'B1', 'C1', 'D2', 'D3', 'D4'], 'A2': ['A1', 'B2', 'C2', 'D2', 'A3', 'A4'], 'A3': ['A1', 'A2', 'B3', 'C3', 'D3', 'A4'], 'A4': ['A1', 'A2', 'A3', 'B4', 'C4', 'D4'], 'B2': ['B1', 'A2', 'C2', 'D2', 'B3', 'B4'], 'B3': ['B1', 'B2', 'A3', 'C3', 'D3', 'B4'], 'B4': ['B1', 'B2', 'B3', 'A4', 'C4', 'D4'], 'C2': ['C1', 'A2', 'B2', 'D2', 'C3', 'C4'], 'C3': ['C1', 'C2', 'A3', 

**Problema do quadrado latino com quadrados já preenchidos**

Para criar um puzzle com quadrados ja preenchidos, é necessário utilizar outro parâmetro da função, chamado dom_quadrado, que
é um dicionário. Atribui-se o valor x à variavel V da seguinte forma: dom_quadrado{'V':x}

E neste exemplo concluímos que há apenas um valor nos dominios iniciais das variáveis que ja estáo preenchidas com um valor. E consequentemente os outros domínios de outras celulas também teem menos valores porcausa da restrição do alldiff para cada linha e coluna

In [94]:
p = quadrado_latino(len=4, dom_quadrado= {'A1': 3, 'B4': 2, 'C2': 1})
print ('variáveis:', p.variables, '\n')
print ('domínios iniciais:', p.domains, '\n')
print ('vizinhos:', p.neighbors, '\n')

variáveis: ['A1', 'B1', 'C1', 'D1', 'A2', 'B2', 'C2', 'D2', 'A3', 'B3', 'C3', 'D3', 'A4', 'B4', 'C4', 'D4'] 

domínios iniciais: {'A1': [3], 'B4': [2], 'C2': [1], 'B1': [1, 4], 'C1': [2, 4], 'D1': [1, 2, 4], 'A2': [2, 4], 'B2': [3, 4], 'D2': [2, 3, 4], 'A3': [1, 2, 4], 'B3': [1, 3, 4], 'C3': [2, 3, 4], 'D3': [1, 2, 3, 4], 'A4': [1, 4], 'C4': [3, 4], 'D4': [1, 3, 4]} 

vizinhos: defaultdict(<class 'list'>, {'A1': ['B1', 'C1', 'D1', 'A2', 'A3', 'A4'], 'B1': ['A1', 'C1', 'D1', 'B2', 'B3', 'B4'], 'C1': ['A1', 'B1', 'D1', 'C2', 'C3', 'C4'], 'D1': ['A1', 'B1', 'C1', 'D2', 'D3', 'D4'], 'A2': ['A1', 'B2', 'C2', 'D2', 'A3', 'A4'], 'A3': ['A1', 'A2', 'B3', 'C3', 'D3', 'A4'], 'A4': ['A1', 'A2', 'A3', 'B4', 'C4', 'D4'], 'B2': ['B1', 'A2', 'C2', 'D2', 'B3', 'B4'], 'B3': ['B1', 'B2', 'A3', 'C3', 'D3', 'B4'], 'B4': ['B1', 'B2', 'B3', 'A4', 'C4', 'D4'], 'C2': ['C1', 'A2', 'B2', 'D2', 'C3', 'C4'], 'C3': ['C1', 'C2', 'A3', 'B3', 'D3', 'C4'], 'C4': ['C1', 'C2', 'C3', 'A4', 'B4', 'D4'], 'D2': ['D1', 'A2',

Resolva o problema com o backtracking sem inferencia, com inferencia, e com uma heurística.

**Backtracking sem inferencia**

In [95]:
#trocar verbose para true para mais detalhes de cada etapa até á solução
backtracking = backtracking_search(p,verbose = False)
print('Afetações = ', backtracking) 

Afetações =  {'A1': 3, 'B1': 1, 'C1': 2, 'D1': 4, 'A2': 2, 'B2': 4, 'C2': 1, 'D2': 3, 'A3': 1, 'B3': 3, 'C3': 4, 'D3': 2, 'A4': 4, 'B4': 2, 'C4': 3, 'D4': 1}


**Backtracking com inferencia**

In [96]:
#temos de criar um novo problema para cada vez que fazemos uma nova solucao porque ja temos uma solucao no problema anterior do problema 
print("Backtracking com inferencia : forward checking")
p2 = quadrado_latino(len=4, dom_quadrado= {'A1': 3, 'B4': 2, 'C2': 1})
b_inf_forward_checking = backtracking_search(p2, inference = forward_checking, verbose = False)
print('Afetações = ',b_inf_forward_checking)

print("\nBacktracking com inferencia : Mantain Arc Consistent")
p3 = quadrado_latino(len=4, dom_quadrado= {'A1': 3, 'B4': 2, 'C2': 1})
b_inf_mac = backtracking_search(p3,inference = mac, verbose = False)
print('Afetações = ',b_inf_mac)

Backtracking com inferencia : forward checking
Afetações =  {'A1': 3, 'B1': 1, 'C1': 2, 'D1': 4, 'A2': 2, 'B2': 4, 'C2': 1, 'D2': 3, 'A3': 1, 'B3': 3, 'C3': 4, 'D3': 2, 'A4': 4, 'B4': 2, 'C4': 3, 'D4': 1}

Backtracking com inferencia : Mantain Arc Consistent
Afetações =  {'A1': 3, 'B1': 1, 'C1': 2, 'D1': 4, 'A2': 2, 'B2': 4, 'C2': 1, 'D2': 3, 'A3': 1, 'B3': 3, 'C3': 4, 'D3': 2, 'A4': 4, 'B4': 2, 'C4': 3, 'D4': 1}


**Backtracking com heuristica**

In [97]:
p4 = quadrado_latino(len=4, dom_quadrado= {'A1': 3, 'B4': 2, 'C2': 1})
b_heur_mrv = backtracking_search(p4,select_unassigned_variable = mrv, verbose=False)
print('Afetações = ',b_heur_mrv)

Afetações =  {'B4': 2, 'C2': 1, 'A1': 3, 'C1': 2, 'A4': 1, 'B1': 1, 'C4': 4, 'B2': 3, 'A2': 4, 'C3': 3, 'D4': 3, 'B3': 4, 'D2': 2, 'A3': 2, 'D1': 4, 'D3': 1}


## Visualização do problema

In [98]:
# Implemente uma função que permita visualizar o quadrado latino, antes e depois de resolvido.

#o r é para qualquer algoritmo com a solucao em formato dict
def display(p = quadrado_latino (), r = backtracking):
    
   #visualizacao do problema quadrado latino no estado inicial:
    print('Quadrado inicial:\n')
    var = p.variables
    n_linhas = []
    for v in var:
        if v[0] == 'A':
            n_linhas.append(v) #o número de A (ou de outra letra qualquer), dá nos o numero de linhas/ o numero de elementos por
            #linha, que vai ser necessário mais à frente. Por isso criei esta lista so com os A, assim à frente chamo a lenght
            #disto para fazer as contas consoante o numero de linhas de cada problema em específico
    
    valores = {}
    for v in var:
        if len(p.domains[v]) == 1:
            #se nos domínios só tiver 1 valor (as variáveis que estão preenchidas desde o início, ou cujo valor, devido a restrições,
            # já está atribuído)
            valores[v] = p.domains.get(v)[0]
            #adiciona ao dicionário valores o valor (único do seu domínio)
        else:
            #caso contrário, o valor das variáveis que têm mais de um valor no domínio fica '-'
            valores[v] = '-'
    
    n = 0 #vai entrar no ciclo while, onde se vai somar 1 
    count = 1 #para fazer print dos números que são a primeira coluna da grelha
    print ('', end = '    ') #só para questão de alinhamento
    for x in range(len(n_linhas)): #x vai até ao nº de linhas/colunas do quadrado
        print(var[x][0], end = ' ') #fazemos print do primeiro carater da variável, que é a letra - cabeçalho da grelha
    print (' ') #para o próximo print aparecer na linha de baixo
    while n < len(var): 
        if (n+1)%(len(n_linhas)) == 0: #queremos ter tantas linhas no print como o numero de linhas do problema, por isso temos 
            #de mudar de linha quando já tivermos n=nº de linhas elementos. Quando n+1(porque começa no zero) for divisivel pelo
            #numero de linhas, o proximo print aparece na linha seguinte
            print (valores[var[n]], end = '\n') #fazemos print do valor que está associado à variável de indíce n, no dicionário valores
        elif n %(len(n_linhas)) == 0: #quando mudamos de linha, o primeiro elemento tem de ser o número, que corresponde ao 
            #segundo carater da variavel (para fazer uma tabela, em que no cruzamento otemos as variáveis - A1, B1, etc)
            print (count, '|', valores[var[n]], end = ' ' )
            count += 1
        else:
            print (valores[var[n]], end = ' ') 
        n += 1    
    
    #visualização do quadrado latino preenchido usando o algoritmo:
    
    print('\nQuadrado final:\n')
    dict_var = r #dicionário com as soluções do algoritmo
    #print para verificar se a visualizacao esta certa
    #print(dict_var)
    
    n_linhas = []
    valores = {}
    for key in dict_var :#As chaves do dicionário, que são as nossas variáveis (A1, B1, etc)
        valores[key] = dict_var[key] #criar um novo dicionário, em que as chaves são as variáveis (A1, B1, etc) e os valores são os valores que a solução atribuiu às variáveis
        if key[0] == 'A': #nº de variaveis cuja letra é A é o numero de linhas/colunas do quadrado. Usando a lenght desta lista vamos conseguir criar a grelha de visualização
            n_linhas.append(key[1]) 
            
     
    n = 0 #vai entrar no ciclo while, onde se vai somar 1 
    count = 1 #para fazer print dos números que são a primeira coluna da grelha
    print ('', end = '    ') #só para questão de alinhamento
    for x in range(len(n_linhas)): #x vai até ao nº de linhas/colunas do quadrado
        print(var[x][0], end = ' ') #fazemos print do primeiro carater da variável, que é a letra - cabeçalho da grelha
    print (' ') #para o próximo print aparecer na linha de baixo        
    while n < len(var): 
        if (n+1)%(len(n_linhas)) == 0: #queremos ter tantas linhas no print como o numero de linhas do problema, por isso temos 
            #de mudar de linha quando já tivermos n=nº de linhas elementos. Quando n+1(porque começa no zero) for divisivel pelo
            #numero de linhas, o proximo print aparece na linha seguinte
            print (valores[var[n]], end = '\n') #fazemos print do valor que está associado à variável de indíce n, no dicionário valores
        elif n %(len(n_linhas)) == 0: #quando mudamos de linha, o primeiro elemento tem de ser o número, que corresponde ao 
            #segundo carater da variavel (para fazer uma tabela, em que no cruzamento otemos as variáveis - A1, B1, etc)
            print (count, '|', valores[var[n]], end = ' ' )
            count += 1
        else:
            print (valores[var[n]], end = ' ') 
        n += 1

In [99]:
print("Backtracking básico")
display(p , backtracking)
print("\nBacktracking com inferencia forward checking")
display(p2 , b_inf_forward_checking)
print("\nBacktracking com inferencia maintain arc consistency")
display(p3 , b_inf_mac)
print("\nBacktracking com heristica MINIMUM-REMAINING-VALUES")
display(p4 , b_heur_mrv)

Backtracking básico
Quadrado inicial:

    A B C D  
1 | 3 - - -
2 | - - 1 -
3 | - - - -
4 | - 2 - -

Quadrado final:

    A B C D  
1 | 3 1 2 4
2 | 2 4 1 3
3 | 1 3 4 2
4 | 4 2 3 1

Backtracking com inferencia forward checking
Quadrado inicial:

    A B C D  
1 | 3 - - -
2 | - - 1 -
3 | - - - -
4 | - 2 - -

Quadrado final:

    A B C D  
1 | 3 1 2 4
2 | 2 4 1 3
3 | 1 3 4 2
4 | 4 2 3 1

Backtracking com inferencia maintain arc consistency
Quadrado inicial:

    A B C D  
1 | 3 - - -
2 | - - 1 -
3 | - - - -
4 | - 2 - -

Quadrado final:

    A B C D  
1 | 3 1 2 4
2 | 2 4 1 3
3 | 1 3 4 2
4 | 4 2 3 1

Backtracking com heristica MINIMUM-REMAINING-VALUES
Quadrado inicial:

    A B C D  
1 | 3 - - -
2 | - - 1 -
3 | - - - -
4 | - 2 - -

Quadrado final:

    A B C D  
1 | 3 1 2 4
2 | 4 3 1 2
3 | 2 4 3 1
4 | 1 2 4 3


## Criação do problema Futoshiki *5x5*

Mostrem que o código está a funcionar, construindo um problema de Futoshiki *5x5*, imprimindo as variáveis, domínios iniciais, e vizinhos. Adicione os comentários necessários. Utilize o [link](https://www.futoshiki.org/) para gerar puzzles e validar a implementação.

In [125]:
#para a criação do problema futoshiki é necessário adicionar as restrições das inegualidades ao problema quadrado_latino
p5 = quadrado_latino(len = 5, dom_quadrado = {'A1':2,'D3':3}, ineq = ['C2 < C3', 'A1 < B1', 'D3 < D4'])
print ('variáveis:', p5.variables, '\n')
print ('domínios iniciais:', p5.domains, '\n')
print ('vizinhos:', p5.neighbors, '\n')
backtracking = backtracking_search(p5,verbose = False)
print('Afetações = ', backtracking)

variáveis: ['A1', 'B1', 'C1', 'D1', 'E1', 'A2', 'B2', 'C2', 'D2', 'E2', 'A3', 'B3', 'C3', 'D3', 'E3', 'A4', 'B4', 'C4', 'D4', 'E4', 'A5', 'B5', 'C5', 'D5', 'E5'] 

domínios iniciais: {'A1': [2], 'D3': [3], 'B1': [1, 3, 4, 5], 'C1': [1, 3, 4, 5], 'D1': [1, 4, 5], 'E1': [1, 3, 4, 5], 'A2': [1, 3, 4, 5], 'B2': [1, 2, 3, 4, 5], 'C2': [1, 2, 3, 4, 5], 'D2': [1, 2, 4, 5], 'E2': [1, 2, 3, 4, 5], 'A3': [1, 4, 5], 'B3': [1, 2, 4, 5], 'C3': [1, 2, 4, 5], 'E3': [1, 2, 4, 5], 'A4': [1, 3, 4, 5], 'B4': [1, 2, 3, 4, 5], 'C4': [1, 2, 3, 4, 5], 'D4': [1, 2, 4, 5], 'E4': [1, 2, 3, 4, 5], 'A5': [1, 3, 4, 5], 'B5': [1, 2, 3, 4, 5], 'C5': [1, 2, 3, 4, 5], 'D5': [1, 2, 4, 5], 'E5': [1, 2, 3, 4, 5]} 

vizinhos: defaultdict(<class 'list'>, {'A1': ['B1', 'C1', 'D1', 'E1', 'A2', 'A3', 'A4', 'A5'], 'B1': ['A1', 'C1', 'D1', 'E1', 'B2', 'B3', 'B4', 'B5'], 'C1': ['A1', 'B1', 'D1', 'E1', 'C2', 'C3', 'C4', 'C5'], 'D1': ['A1', 'B1', 'C1', 'E1', 'D2', 'D3', 'D4', 'D5'], 'E1': ['A1', 'B1', 'C1', 'D1', 'E2', 'E3', 'E4',

Resolva o problema com o backtracking sem inferencia, com inferencia, e com uma heurística. Até que dimensão consegue resolver o problema em menos de 1 minuto?

In [126]:
# código de aplicação dos algoritmos
#trocar verbose para true para mais detalhes de cada etapa até á solução
import timeit

dim = 8

print("Backtracking simples")
p5 = quadrado_latino(len = dim, dom_quadrado = {'A1':2,'D3':3}, ineq = ['C2 < C3', 'A1 < B1', 'D3 < D4'])
start1 = timeit.default_timer()
backtracking = backtracking_search(p5,verbose = False)
stop1 = timeit.default_timer()
print('Afetações = ', backtracking) 
tempo1 = stop1 - start1
print("Tempo: ", tempo1)

print("\nBacktracking com inferencia : forward checking")
p6 = quadrado_latino(len = dim, dom_quadrado = {'A1':2,'D3':3}, ineq = ['C2 < C3', 'A1 < B1', 'D3 < D4'])
start2 = timeit.default_timer()
b_inf_forward_checking = backtracking_search(p6, inference = forward_checking, verbose = False)
stop2 = timeit.default_timer()
print('Afetações = ',b_inf_forward_checking)
tempo2 = stop2 - start2
print("Tempo: ", tempo2)

print("\nBacktracking com inferencia : Mantain Arc Consistent")
p7 = quadrado_latino(len = dim, dom_quadrado = {'A1':2,'D3':3}, ineq = ['C2 < C3', 'A1 < B1', 'D3 < D4'])
start3 = timeit.default_timer()
b_inf_mac = backtracking_search(p7,inference = mac, verbose = False)
stop3 = timeit.default_timer()
print('Afetações = ',b_inf_mac)
tempo3 = stop3 - start3
print("Tempo: ", tempo3)

print("\nBacktracking com heuristica : mrv")
p8 = quadrado_latino(len = dim, dom_quadrado = {'A1':2,'D3':3}, ineq = ['C2 < C3', 'A1 < B1', 'D3 < D4'])
start4 = timeit.default_timer()
b_heur_mrv = backtracking_search(p8,select_unassigned_variable = mrv, verbose=False)
stop4 = timeit.default_timer()
print('Afetações = ',b_heur_mrv)
tempo4 = stop4 - start4
print("Tempo: ", tempo4)

#RESULTADOS

# Em ambos backtracking sem e com inferencia, a partir da dimensão 10 (len = 10), o código demora mais de 1 minuto a executar.
# Por outro lado, no backtracking com uma heuristica, qualquer dimensão acima de 5 demora mais de 1 minuto a executar. 
#Note que depende também da performance de cada computador


Backtracking simples
Afetações =  {'A1': 2, 'B1': 3, 'C1': 1, 'D1': 4, 'E1': 5, 'F1': 6, 'G1': 7, 'H1': 8, 'I1': 9, 'A2': 1, 'B2': 2, 'C2': 3, 'D2': 5, 'E2': 4, 'F2': 7, 'G2': 6, 'H2': 9, 'I2': 8, 'A3': 4, 'B3': 1, 'C3': 5, 'D3': 3, 'E3': 2, 'F3': 8, 'G3': 9, 'H3': 6, 'I3': 7, 'A4': 3, 'B4': 4, 'C4': 2, 'D4': 6, 'E4': 1, 'F4': 9, 'G4': 8, 'H4': 7, 'I4': 5, 'A5': 5, 'B5': 6, 'C5': 7, 'D5': 9, 'E5': 8, 'F5': 3, 'G5': 2, 'H5': 1, 'I5': 4, 'A6': 6, 'B6': 5, 'C6': 8, 'D6': 7, 'E6': 9, 'F6': 4, 'G6': 1, 'H6': 2, 'I6': 3, 'A7': 7, 'B7': 8, 'C7': 9, 'D7': 1, 'E7': 3, 'F7': 2, 'G7': 5, 'H7': 4, 'I7': 6, 'A8': 8, 'B8': 9, 'C8': 6, 'D8': 2, 'E8': 7, 'F8': 5, 'G8': 4, 'H8': 3, 'I8': 1, 'A9': 9, 'B9': 7, 'C9': 4, 'D9': 8, 'E9': 6, 'F9': 1, 'G9': 3, 'H9': 5, 'I9': 2}
Tempo:  0.2615339000003587

Backtracking com inferencia : forward checking
Afetações =  {'A1': 2, 'B1': 3, 'C1': 1, 'D1': 4, 'E1': 5, 'F1': 6, 'G1': 7, 'H1': 8, 'I1': 9, 'A2': 1, 'B2': 2, 'C2': 3, 'D2': 5, 'E2': 4, 'F2': 7, 'G2': 6, 'H2

KeyboardInterrupt: 

## Visualização do problema

In [129]:
# Implemente uma função que permita visualizar o quadrado latino, antes e depois de resolvido.


#o r é para qualquer algoritmo com a solucao em formato dict
def display_f(p = quadrado_latino (), r = backtracking, ineq = ['C2 < C3', 'A1 < B1', 'D4 > D3']):
    letras = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    desigualdades = []
        
    for i in ineq:
        i = i.split(' ')
        desigualdades.append(i)
        
    menor_que_seguinte = []
    maior_que_seguinte = [] 
    menor_que_baixo = []
    maior_que_baixo = []
    for d in desigualdades:
        if d[0][1] == d[2][1] and letras.index(d[0][0]) == letras.index(d[2][0]) - 1 and d[1] == '<': #número igual e a letra do primeiro elemento é a anterior à letra do segundo (mesma linha e o 1º elemento está à esquerda do 2º)
            menor_que_seguinte.append(d[0])
        if d[0][1] == d[2][1] and letras.index(d[0][0]) == letras.index(d[2][0]) + 1 and d[1] == '<':
            maior_que_seguinte.append(d[2])
        if d[0][1] == d[2][1] and letras.index(d[0][0]) == letras.index(d[2][0]) - 1 and d[1] == '>':
            maior_que_seguinte.append(d[0])   
        if d[0][1] == d[2][1] and letras.index(d[0][0]) == letras.index(d[2][0]) + 1 and d[1] == '>':
            menor_que_seguinte.append(d[2])
        if d[0][0] == d[2][0] and int(d[0][1]) == int(d[2][1]) - 1 and d[1] == '<':
            menor_que_baixo.append(d[0])
        if d[0][0] == d[2][0] and int(d[0][1]) == int(d[2][1]) + 1 and d[1] == '<':
            maior_que_baixo.append(d[2])
        if d[0][0] == d[2][0] and int(d[0][1]) == int(d[2][1]) - 1 and d[1] == '>':
            maior_que_baixo.append(d[0])
        if d[0][0] == d[2][0] and int(d[0][1]) == int(d[2][1]) + 1 and d[1] == '>':
            menor_que_baixo.append(d[2])    
   
            
   #visualizacao do problema quadrado latino no estado inicial:
    print('Quadrado inicial:\n')
    var = p.variables
    n_linhas = []
    for v in var:
        if v[0] == 'A':
            n_linhas.append(v) #o número de A (ou de outra letra qualquer), dá nos o numero de linhas/ o numero de elementos por
            #linha, que vai ser necessário mais à frente. Por isso criei esta lista so com os A, assim à frente chamo a lenght
            #disto para fazer as contas consoante o numero de linhas de cada problema em específico
    
    valores = {}
    desigualdades = {}
    for v in var:
        if len(p.domains[v]) == 1:
            #se nos domínios só tiver 1 valor (as variáveis que estão preenchidas desde o início, ou cujo valor, devido a restrições,
            # já está atribuído)
            valores[v] = p.domains.get(v)[0]
            #adiciona ao dicionário valores o valor (único do seu domínio)
        else:
            #caso contrário, o valor das variáveis que têm mais de um valor no domínio fica '-'
            valores[v] = '-'
        if v in menor_que_baixo:
            desigualdades[v] = '<'
        elif v in maior_que_baixo:
            desigualdades[v] = '>'
        else:
            desigualdades[v] = ' '
            
    x = 0
    y = len(n_linhas)
    n = 0 #vai entrar no ciclo while, onde se vai somar 1 
    count = 1 #para fazer print dos números que são a primeira coluna da grelha
    print ('', end = '    ') #só para questão de alinhamento
    for x in range(len(n_linhas)):#x vai até ao nº de linhas/colunas do quadrado
        
        print(var[x][0], end = '    ') #fazemos print do primeiro carater da variável, que é a letra - cabeçalho da grelha
    print (' ') #para o próximo print aparecer na linha de baixo
    
    while n < len(var): 
        if (n+1)%(len(n_linhas)) == 0: #queremos ter tantas linhas no print como o numero de linhas do problema, por isso temos 
            #de mudar de linha quando já tivermos n=nº de linhas elementos. Quando n+1(porque começa no zero) for divisivel pelo
            #numero de linhas, o proximo print aparece na linha seguinte
            if var[n] in maior_que_seguinte:
                print (valores[var[n]], '> ', end = '\n')#fazemos print do valor que está associado à variável de indíce n, no dicionário valores
            elif var[n] in menor_que_seguinte:
                 print (valores[var[n]], '< ', end = '\n')
            else:
                print (valores[var[n]], '  ',end = '\n')
            
            print ([desigualdades[var[n]] for n in range(x, y)])

        

        
        elif n %(len(n_linhas)) == 0: #quando mudamos de linha, o primeiro elemento tem de ser o número, que corresponde ao 
            #segundo carater da variavel (para fazer uma tabela, em que no cruzamento otemos as variáveis - A1, B1, etc)
            if var[n] in maior_que_seguinte:
                print (count, '|', valores[var[n]], '> ', end = ' ' )
            elif var[n] in menor_que_seguinte:
                print (count, '|', valores[var[n]], '< ', end = ' ' )
            else:
                print (count, '|', valores[var[n]], '  ', end = ' ' )
            count += 1
        else:
            if var[n] in maior_que_seguinte:
                print (valores[var[n]], '> ', end = ' ') 
            elif var[n] in menor_que_seguinte:
                print (valores[var[n]], '< ', end = ' ') 
            else:
                print (valores[var[n]],'  ', end = ' ') 
        n += 1
        x += int(len(n_linhas))
    
    #visualização do quadrado latino preenchido usando o algoritmo:
    
    print('\nQuadrado final:\n')
    dict_var = r #dicionário com as soluções do algoritmo
    #print para verificar se a visualizacao esta certa
    #print(dict_var)
    
    n_linhas = []
    valores = {}
    for key in dict_var :#As chaves do dicionário, que são as nossas variáveis (A1, B1, etc)
        valores[key] = dict_var[key] #criar um novo dicionário, em que as chaves são as variáveis (A1, B1, etc) e os valores são os valores que a solução atribuiu às variáveis
        if key[0] == 'A': #nº de variaveis cuja letra é A é o numero de linhas/colunas do quadrado. Usando a lenght desta lista vamos conseguir criar a grelha de visualização
            n_linhas.append(key[1]) 
            
     
    n = 0 #vai entrar no ciclo while, onde se vai somar 1 
    count = 1 #para fazer print dos números que são a primeira coluna da grelha
    print ('', end = '    ') #só para questão de alinhamento
    for x in range(len(n_linhas)): #x vai até ao nº de linhas/colunas do quadrado
        print(var[x][0], end = '    ') #fazemos print do primeiro carater da variável, que é a letra - cabeçalho da grelha
    print (' ') #para o próximo print aparecer na linha de baixo        
    while n < len(var): 
        if (n+1)%(len(n_linhas)) == 0: #queremos ter tantas linhas no print como o numero de linhas do problema, por isso temos 
            #de mudar de linha quando já tivermos n=nº de linhas elementos. Quando n+1(porque começa no zero) for divisivel pelo
            #numero de linhas, o proximo print aparece na linha seguinte
            if var[n] in maior_que_seguinte:
                print (valores[var[n]],'> ', end = '\n')
            elif var[n] in menor_que_seguinte:
                print (valores[var[n]],'< ', end = '\n')
            else:
                print (valores[var[n]], '  ', end = '\n') #fazemos print do valor que está associado à variável de indíce n, no dicionário valores
        elif n %(len(n_linhas)) == 0: #quando mudamos de linha, o primeiro elemento tem de ser o número, que corresponde ao 
            #segundo carater da variavel (para fazer uma tabela, em que no cruzamento otemos as variáveis - A1, B1, etc)
            if var[n] in maior_que_seguinte:
                print (count, '|', valores[var[n]], '> ', end = ' ' )
            elif var[n] in menor_que_seguinte:
                print (count, '|', valores[var[n]], '< ', end = ' ' )
            else:  
                print (count, '|', valores[var[n]], '  ', end = ' ' )
            count += 1
        else:
            if var[n] in maior_que_seguinte:
                print (valores[var[n]], '>  ', end = ' ')
            elif var[n] in menor_que_seguinte:
                print (valores[var[n]], '<  ', end = ' ')
            else: 
                print (valores[var[n]], '  ', end = ' ') 
        n += 1    
        
    print ([desigualdades[var[n]] for n in range(4)])


In [130]:
display_f(p , backtracking)

Quadrado inicial:

    A    B    C    D     
1 | 3 <  -    -    -   
[]
2 | -    -    1    -   
[]
3 | -    -    -    -   
[]
4 | -    2    -    -   
[]

Quadrado final:

    A    B    C    D    A    B    C    D    A     
1 | 2 <  3    1    4    1    2    3    5    4   
2 | 1    5    3    3    4    2    6    [' ', ' ', ' ', ' ']


**Teste com puzzle "Easy" sem espaços preenchidos**

In [136]:
p5 = quadrado_latino(len=4, dom_quadrado={}, ineq=['A1 < B1', 'C1 > C2', 'A2 < B2', 'C3 > C4', 'C4 > D4', 'A4 < B4'])

# print('variáveis:', p.variables, '\n')
# print('domínios iniciais:', p.domains, '\n')
# print('vizinhos:', p.neighbors, '\n')


#display(p4, backtracking)
backtracking = backtracking_search(p5, verbose=False)
print('Assignment p = ', backtracking)

display(p5, backtracking)

Assignment p =  {'A1': 1, 'B1': 2, 'C1': 4, 'D1': 3, 'A2': 2, 'B2': 3, 'C2': 1, 'D2': 4, 'A3': 4, 'B3': 1, 'C3': 3, 'D3': 2, 'A4': 3, 'B4': 4, 'C4': 2, 'D4': 1}
Quadrado inicial:

    A B C D  
1 | - - - -
2 | - - - -
3 | - - - -
4 | - - - -

Quadrado final:

    A B C D  
1 | 1 2 4 3
2 | 2 3 1 4
3 | 4 1 3 2
4 | 3 4 2 1


![title](TESTE4BEFORE.png)

![title](TESTE4AFTER.png)

**Teste com puzzle "Easy" com espaços preenchidos**

In [131]:
p3 = quadrado_latino(len=4, dom_quadrado={'C1' : 2, 'D1' : 4, 'D2' :2 }, ineq=['A1 > B1', 'B3 > B2' , 'B4 > C4'])

# print('variáveis:', p.variables, '\n')
# print('domínios iniciais:', p.domains, '\n')
# print('vizinhos:', p.neighbors, '\n')


#display(p3, backtracking)
backtracking = backtracking_search(p3, verbose=False)
print('Assignment p = ', backtracking)

display(p3, backtracking)


Assignment p =  {'A1': 3, 'B1': 1, 'C1': 2, 'D1': 4, 'A2': 1, 'B2': 3, 'C2': 4, 'D2': 2, 'A3': 2, 'B3': 4, 'C3': 3, 'D3': 1, 'A4': 4, 'B4': 2, 'C4': 1, 'D4': 3}
Quadrado inicial:

    A B C D  
1 | - - 2 4
2 | - - - 2
3 | - - - -
4 | - - - -

Quadrado final:

    A B C D  
1 | 3 1 2 4
2 | 1 3 4 2
3 | 2 4 3 1
4 | 4 2 1 3


![title](teste2beforee.png)

![title](teste2after.png)

**Teste com puzzle "Tricky" com espaços preenchidos**

In [134]:
p4 = quadrado_latino(len=4, dom_quadrado={'B4': 1, 'D4' : 2}, ineq=['A1 > B1', 'B3 > C3', 'D4 > D3'])

# print('variáveis:', p.variables, '\n')
# print('domínios iniciais:', p.domains, '\n')
# print('vizinhos:', p.neighbors, '\n')

#display(p4, backtracking)
backtracking = backtracking_search(p4, verbose=False)
print('Assignment p = ', backtracking)

display(p4, backtracking)

Assignment p =  {'A1': 4, 'B1': 2, 'C1': 1, 'D1': 3, 'A2': 1, 'B2': 3, 'C2': 2, 'D2': 4, 'A3': 2, 'B3': 4, 'C3': 3, 'D3': 1, 'A4': 3, 'B4': 1, 'C4': 4, 'D4': 2}
Quadrado inicial:

    A B C D  
1 | - - - -
2 | - - - -
3 | - - - -
4 | - 1 - 2

Quadrado final:

    A B C D  
1 | 4 2 1 3
2 | 1 3 2 4
3 | 2 4 3 1
4 | 3 1 4 2


![title](teste3before.png)

![title](teste3aafter.png)